In [684]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [685]:
import datetime
from copy import deepcopy

import argparse
import itertools
import json
import os
from glob import glob
import torch
from tqdm.auto import tqdm

from statsmodels.tools import add_constant
from numpy.polynomial import chebyshev
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', 50) 
from torch import nn
import matplotlib.pyplot as plt

from run_model import generate_parser, main
from pipeline.pipeline import (
    compute_inverse_design,
    loss_fn,
    loss_fn_qiv,
    null_regularizer,
    stopping_criterion,
    train_loop,
    train_step,
    transform_endogenous_wrapper,
    weight_fn,
    l1_regularizer
)

from architecture.architectures import (
    Nonparametric,
    PartiallyAdditive,
    PartiallyAdditiveWithSpline,
    PartiallyLinear,
)

from pipeline.callbacks import callback, log_callback, tensorboard_callback, writer

## Variables and Data Types

In [686]:
organic_strawb.dtypes

q_own             float64
q_other           float64
s_own             float64
s_other           float64
p_own             float64
p_other           float64
p_out             float64
p_out_avg         float64
x_own             float64
x_other           float64
x_month           float64
x_usda_lettuce    float64
x_outf            float64
income            float64
z_own             float64
z_other           float64
z_out             float64
p_change            int64
spot_own          float64
spot_other        float64
p_own_nonn        float64
p_other_nonn      float64
dtype: object

In [687]:
var_definitions = {
    "s_own": "market share (for a given store/week combination) of product 1 (=non-organic strawberries)",
    "s_other": "market share of product 2  (=organic strawberries)",
    "p_own": "price per pound of product 1",
    "p_other": "price per pound of product 2",
    "p_out_avg": "price per pound of outside option (=other fruit)",
    "x_usda_lettuce": "proxy for taste for organic products at store",  # x_org^1
    "x_outf": "proxy for richness of the outside option",  # x_str^1 ?
    "income": "income at zipcode level",  # x^2
    "spot_own": "spot price (a measure of marginal cost) for product 1",
    "spot_other": "spot price for product 2",
    "z_own": "Hausman IV (price of same product in neighboring markets) for product 1",
    "z_other": "Hausman IV (price of same product in neighboring markets) for product 2",
    "z_out": "Hausman IV (price of same product in neighboring markets) for outside option (other fruit)",
}

## Some Basic Data Preprocessing

In [688]:
strawb = pd.read_csv("data_1.csv")
organic_strawb = pd.read_csv("data_2.csv")


# drop redundant columns and columns with 0 variation
string_cols =  list(strawb.columns[:5])+list(strawb.columns[-9:])
redund_strawb = string_cols + ['usda'] + ['x_usda_lettuce_2'] + ['x_outf_2']
redund_org_strawb = string_cols+ ['usda'] + ['x_usda_lettuce_2'] + ['x_outf_2']
strawb = strawb.drop(columns = redund_strawb)
organic_strawb = organic_strawb.drop(columns = redund_org_strawb)

# transform variables in logs so that by finding 
# the average derivative we obtain our functional
# of interest - elasticity
strawb['q_own'] = np.log(strawb['q_own'])
strawb['q_other'] = np.log(strawb['q_other'])
strawb['p_own'] = np.log(strawb['p_own'])
strawb['p_other'] = np.log(strawb['p_other'])
organic_strawb['q_own'] = np.log(organic_strawb['q_own'])
organic_strawb['q_other'] = np.log(organic_strawb['q_other'])
organic_strawb['p_own'] = np.log(organic_strawb['p_own'])
organic_strawb['p_other'] = np.log(organic_strawb['p_other'])

We will be estimating the following equation using the NPIV model:
$log(q_{jt}) = h(log(p_{jt}),log(p_{-jt}), x_{jt},m_{t})$ \
Our instruments will be Hausman type IV that act as a proxy of marginal cost shifters. This is not the model \
used in Compiani 2021, but a simplified version since the NPIV approach with NN as a first stage estimator \
performed poorly with his estimating equation.

## Useful Functions

In [689]:
## different instrument transforms 

# no interactions
def random_fourier_basis(x, scale=5, d=60, seed=1):
    n, d_in = x.shape
    rng = np.random.RandomState(seed)
    w = rng.randn(d_in, d) * scale
    b = rng.rand(1, d) * 2 * np.pi
    return add_constant(np.cos(x @ w + b))

# with interactions
def chebyshev_expansion(var, max_degree=3, intercept=False):
    expansion = []
    for i in range(1 - int(intercept), max_degree + 1):
        c = np.zeros(max_degree + 1)
        c[i] = 1
        expansion.append(chebyshev.chebval(2 * var - 1, c))
    return np.array(expansion).T

## data preparation 

def wrangle_data_quantity_regr(organic=False, chebyshev=False):
    
    # specify what column is the outcome variable,
    # what columns are endogenous (they enter the unknown NPIV h() function)
    # and what columns are instruments
    y = "q_own"
    endo_cols = ["p_own", "p_other", 'x_usda_lettuce','x_outf', "income"] # order of variables is important
    instrument_cols = [
        'x_usda_lettuce',
        'x_outf',
        "income",
        "spot_own",
        "spot_other",
        "z_own",
        "z_other",
        "z_out",
    ]

    data = organic_strawb if organic else strawb

    response = data[y].values.copy()[:, None]
    endogenous = data[endo_cols].values.copy()
    instrument = data[instrument_cols].values.copy()

    # form an instrument basis that is used to project
    # on an instrument space and obtain conditional expectation
    if chebyshev:
        transformed_instrument = add_constant(
            np.hstack(
                [chebyshev_expansion(var, intercept=False) for var in instrument.T]
                + [
                    (i * j)[:, None]
                    for i in instrument.T
                    for j in instrument.T
                    if (i != j).any()
                ]
            )
        )
    else:
        transformed_instrument = random_fourier_basis(instrument)

    # organises our data objects as a dictionary
    # that will be an input in our model
    npvec = {
        "response": response,
        "endogenous": endogenous,
        "instrument": instrument,
        "transformed_instrument": transformed_instrument,
        "inverse_design_instrument": np.linalg.pinv(
            transformed_instrument.T
            @ transformed_instrument
            / len(transformed_instrument),
            rcond=1e-5,
        ),
    }
    
    # prints the rank of the design matrix
    print(np.linalg.matrix_rank(npvec["inverse_design_instrument"]), npvec["inverse_design_instrument"].shape)

    # transforms data dictionary into pytorch format
    torchvec = {k: torch.as_tensor(v).float() for k, v in npvec.items()}
    return npvec, torchvec


## derivatives & elasticities

# uses pytorch to find a partial derivative of the model in a certain variable
def get_deriv(model, torchvec, which="p_own"):
    lst = "p_own p_other x_usda_lettuce x_outf income".split()
    index = lst.index(which)
    return model.get_derivatives(torchvec['endogenous'], index=index).numpy().flatten()

# # this one is for derivatives at a particular point - works poorly
# def get_deriv_elast(model, torchvec, which="s_own"):
#     lst = "s_own s_other p_own p_other".split()
#     index = lst.index(which)
#     return model.get_derivatives_for_elast(torchvec, index=index)



# applies get_deriv to find own and cross price elasticities 
def get_elast_quantity_regr(own_model, own_torchvec):
    top = get_deriv(own_model, own_torchvec, "p_own")
    p_own = own_torchvec["endogenous"][:, 0].numpy().flatten()
    
    return top, (p_own, top)

def get_cross_elast_quantity_regr(own_model, own_torchvec):
    top = get_deriv(own_model, own_torchvec, "p_other")
    p_other = own_torchvec["endogenous"][:, 1].numpy().flatten()
    
    return top, (p_other,  top)

def fit_model(torchvec, model_kwargs, optimizer_kwargs, name, max_iter=3000, with_weights = False):
    
    # load the specific model class that is suitable
    # for this problem. It could be one of the following:
    # Nonparametric, PartiallyAdditive,
    # PartiallyAdditiveWithSpline, PartiallyLinear.
    # Here I use the most general Nonparametric class
    
    model = Nonparametric(
        input_dim=torchvec["endogenous"].shape[1], **model_kwargs
    )
    optimizer = torch.optim.Adam(model.parameters(), **optimizer_kwargs)

    #training
    
    # basic model
    for i in tqdm(range(max_iter)):
        outcome = train_step(
            model,
            optimizer,
            response=torchvec["response"],
            endogenous=torchvec["endogenous"],
            transformed_instrument=torchvec["transformed_instrument"],
            inverse_design_instrument=torchvec["inverse_design_instrument"]
        )
        
        # stores intermediate results of training
        # could be useful to track how the model is learning
        if i % 20 == 0:
            result_dict = {
                item: get_deriv(model, torchvec, which=item).mean()
                for item in "p_own p_other x_usda_lettuce x_outf income".split()
            }
            result_dict.update(
                {"loss": outcome["loss"], "grad_norm": outcome["grad_norm"]}
            )

            tensorboard_callback(
                i, cache_df=pd.Series(result_dict).to_frame().T, name=name,
            )
            
    # efficiency weighted model        
    if with_weights:
            
        weights = weight_fn(
            prediction=model(torchvec["endogenous"]),
            truth=torchvec["response"],
            basis=torchvec["instrument"],
            n_neighbors=5,
        )
        torchvec["weights"] = weights
        
        model_w_weights = Nonparametric(
        input_dim=torchvec["endogenous"].shape[1], **model_kwargs)
        optimizer = torch.optim.Adam(model_w_weights.parameters(), **optimizer_kwargs)
        
        for i in tqdm(range(max_iter)):
            outcome = train_step(
                model_w_weights,
                optimizer,
                response=torchvec["response"],
                endogenous=torchvec["endogenous"],
                transformed_instrument=torchvec["transformed_instrument"],
                inverse_design_instrument=torchvec["inverse_design_instrument"],
                weights=weights
            )
            
        # stores intermediate results of training
        # could be useful to track how the model is learning
        if i % 20 == 0:
            result_dict = {
                item: get_deriv(model_w_weights, torchvec, which=item).mean()
                for item in "p_own p_other x_usda_lettuce x_outf income".split()
            }
            result_dict.update(
                {"loss": outcome["loss"], "grad_norm": outcome["grad_norm"]}
            )

            tensorboard_callback(
                i, cache_df=pd.Series(result_dict).to_frame().T, name=name,
            )
        return model, model_w_weights, optimizer
    
    else: 
        
        return model, optimizer

# Needs more comments!!    
# This function computes standard errors 
# See the paper and Github Repo for more details
def compute_se(
    torchvec,
    model,
    weights=None,
    inefficient_derivative=None,
    inefficient_prediction=None,
    order=1,
    weighting=False,
):
    try:
        se_nonpar = None
        dim_x_tilde = torchvec["endogenous"].shape[1] - 3
        if hasattr(model, "get_standard_error_nonparametric"):
            (
                tf_endogenous,
                tf_endogenous_gradient,
                transformed_instrument,
                inverse_design,
            ) = transform_endogenous_wrapper(
                torchvec["endogenous"],
                torchvec["instrument"],
                torchvec["transformed_instrument"],
                True,
                dim_x_tilde,
                order,
                interact_x=True,
            )

            inv_variance = weight_fn(
                prediction=model(torchvec["endogenous"]),
                truth=torchvec["response"],
                basis=transformed_instrument,
                inverse_design=inverse_design,
            )

            if weighting:
                filtered, Gamma = model.forward_filter_residuals(
                    endogenous=torchvec["endogenous"],
                    response=torchvec["response"],
                    inefficient_derivative=inefficient_derivative,
                    inefficient_prediction=inefficient_prediction,
                    weights=weights,
                    basis=transformed_instrument,
                    inverse_design=inverse_design,
                )

                se_nonpar = model.get_standard_error_nonparametric(
                    filtered,
                    Gamma,
                    tf_endogenous,
                    tf_endogenous_gradient,
                    transformed_instrument,
                    inv_variance,
                    inverse_design,
                )
            else:
                se_nonpar = model.get_standard_error_nonparametric(
                    model.get_derivatives(torchvec["endogenous"]),
                    0,
                    tf_endogenous,
                    tf_endogenous_gradient,
                    transformed_instrument,
                    1,
                    inverse_design,
                    weighting=False,
                    residuals=torchvec["response"] - model(torchvec["endogenous"]),
                )

        (
            tf_endogenous,
            tf_endogenous_gradient,
            transformed_instrument,
            inverse_design,
        ) = transform_endogenous_wrapper(
            torchvec["endogenous"],
            torchvec["instrument"],
            torchvec["transformed_instrument"],
            False,
            dim_x_tilde,
            order,
        )

        inverse_variance = weight_fn(
            prediction=model(torchvec["endogenous"]),
            truth=torchvec["response"],
            basis=transformed_instrument,
            inverse_design=inverse_design,
        )
        se = model.get_standard_error(
            endogenous_of_interest=torchvec["endogenous"][:, [0]],
            transformed_endogenous=tf_endogenous,
            transformed_instrument=transformed_instrument,
            inverse_variance=inverse_variance,
            inverse_design_instrument=inverse_design,
        )

        return se, se_nonpar
    except RuntimeError:
        return np.nan, np.nan


Here we prepare the data for training and specify the instrument basis. As an output we get torch dictionary with data that will be used in training. Also, the rank of the design matrix is printed.

In [690]:
npvec_strawb, torchvec_strawb = wrangle_data_quantity_regr(organic=False, chebyshev=False)
npvec_organic, torchvec_organic = wrangle_data_quantity_regr(organic=True, chebyshev=False)

61 (61, 61)
61 (61, 61)


## Modelling

Next, it is necessary to specify the NN architecture. Here the user has multiple options that affect the performance. The most obvious ones include the number of layers (depth) and the width of each layer. Also, activation function can matter in training.

In [691]:
model_kwargs = dict(depth=1, width=20, hidden_activation=torch.nn.ReLU)

Here we train models for organic and non-organic strawberry. Results can depend significantly on the learning rate, and the number of iterations. There is no clear way theoretically on how to choose those hyperparameters. Important note is that there are various options for model training within the model class. For instance, instead of using identity weighting one can efficiently weight the data using some initial consistent estimator. Also, instead of using initial moment condition one can work with orthogonalised moments to improve efficiency. I encourage you to check the paper and **pipeline** and **architecture** folders to learn more about this.

In [692]:
# Train models on our data
model_strawb, opt_strawb = fit_model(
    torchvec_strawb,
    model_kwargs,
    dict(lr=0.001, weight_decay=1e-6),
    max_iter=2000,
    name=f"strawb.{datetime.datetime.now()}",
    with_weights = False
)
model_organic, opt_organic = fit_model(
    torchvec_organic,
    model_kwargs,
    dict(lr=0.001, weight_decay=1e-6),
    max_iter=2000,
    name=f"organic.{datetime.datetime.now()}",
)

## Standard errors

The pipeline package provides a function *compute_se* to obtain standard errors for the estimated model.

In [693]:
# This function computes standard errors
# at this point it requires variable of interest to be the first 
# in torchvec 
se, se_nonparam = compute_se(torchvec_strawb, model_strawb)
se_org, se_nonparam_org = compute_se(torchvec_organic, model_organic)

## Results

In [694]:
elast, ingredients = get_elast_quantity_regr(
    model_strawb, torchvec_strawb)
cross_elast, ingredients_cross = get_cross_elast_quantity_regr(model_strawb, torchvec_strawb)

In [695]:
elast_org, ingredients_org = get_elast_quantity_regr(
    model_organic, torchvec_organic)
cross_elast_org, ingredients_cross_org = get_cross_elast_quantity_regr(
    model_organic, torchvec_organic)

### Non-organic elasticities 

In [696]:
print(f"Elast: {np.format_float_positional(np.mean(elast),2)}")
print(f"Cross Elast: {np.format_float_positional(np.mean(cross_elast), 2)}")

Elast: -2.68
Cross Elast: -1.68


### Organic elasticites

In [697]:
print(f"Elast: {np.format_float_positional(np.mean(elast_org),2)}")
print(f"Cross Elast: {np.format_float_positional(np.mean(cross_elast_org),2)}")

Elast: -2.14
Cross Elast: 0.19


One can see that organic strawberries are less price elastic

In [ ]:
# pd.Series(elast).describe(percentiles=np.linspace(0, 1, 51))